In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape(original, do not use)/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
# Double check before running agin


In [7]:
df=pd.read_csv('../../Data/Taxi Data/yellow_15/yellow_tripdata_2015-08.csv')

In [8]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

df.info()

df2=df.head(100)

df2.columns

In [9]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x['dropoff_longitude']), 
                                             float(x['dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 677.9874284267426 seconds ---


In [10]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x['pickup_longitude']), 
                                             float(x['pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 685.1172714233398 seconds ---


In [11]:
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dropoff_geometry,pickup_geometry
0,2,2015-08-01 00:00:15,2015-08-01 00:36:21,1,7.22,-73.999809,40.743340,1,N,-73.942848,...,2,29.5,0.5,0.5,0.00,0.0,0.3,30.80,POINT (-73.94284820556641 40.80662155151367),POINT (-73.99980926513672 40.74333953857422)
1,1,2015-08-01 00:00:16,2015-08-01 00:14:52,1,2.30,-73.977043,40.774902,1,N,-73.978256,...,1,12.0,0.5,0.5,2.93,0.0,0.3,16.23,POINT (-73.97825622558594 40.74986267089844),POINT (-73.97704315185547 40.77490234375)
2,1,2015-08-01 00:00:16,2015-08-01 00:06:30,1,1.50,-73.959122,40.775127,1,N,-73.980392,...,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,POINT (-73.98039245605469 40.78231430053711),POINT (-73.95912170410156 40.77512741088867)


In [12]:
df2_start=df[['passenger_count','trip_distance','fare_amount','mta_tax','tip_amount','tolls_amount','pickup_geometry']]

In [13]:
df2_end=df[['dropoff_geometry']]

In [14]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 90.7814154624939 seconds ---


In [15]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 90.65503478050232 seconds ---


In [16]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 915.7237129211426 seconds ---


In [ ]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

In [ ]:
geo_processing_start

In [ ]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [ ]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [ ]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [ ]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [ ]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [ ]:
merge

In [ ]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2015-08-processed.csv')
complete

In [ ]:
print("--- %s seconds ---" % (time.time() - full_time))